In [0]:
%sql
CREATE DATABASE IF NOT EXISTS saas_bronze;
CREATE DATABASE IF NOT EXISTS saas_silver;
CREATE DATABASE IF NOT EXISTS saas_gold;


This step creates separate databases for each Medallion layer:

Bronze → Raw ingested data

Silver → Cleaned and transformed data

Gold → Business-ready analytics

In [0]:
%sql
CREATE TABLE IF NOT EXISTS saas_bronze.feature_usage_logs (
    user_id STRING,
    session_id STRING,
    feature_name STRING,
    event_type STRING,
    event_timestamp TIMESTAMP
)
USING DELTA;


This table stores raw product interaction logs:

Captures user behavior (clicks, views)

Stored in Delta format for reliability

No transformations applied yet

This is the raw behavioral dataset.

In [0]:
%sql
INSERT INTO saas_bronze.feature_usage_logs VALUES
-- Highly engaged user
('U1001','S1','Dashboard','click','2026-01-01 10:00:00'),
('U1001','S1','Reports','view','2026-01-01 10:02:00'),
('U1001','S1','Analytics','click','2026-01-01 10:05:00'),
('U1001','S2','Dashboard','click','2026-01-03 11:00:00'),
('U1001','S2','Settings','click','2026-01-03 11:04:00'),

-- Medium engagement user
('U1002','S3','Dashboard','click','2026-01-02 09:00:00'),
('U1002','S3','Dashboard','click','2026-01-02 09:05:00'),
('U1002','S4','Reports','view','2026-01-05 14:00:00'),

-- Low engagement user
('U1003','S5','Dashboard','click','2026-01-04 15:00:00'),

-- Trial user
('U1004','S6','Reports','view','2026-01-06 16:00:00'),
('U1004','S6','Reports','view','2026-01-06 16:03:00');


num_affected_rows,num_inserted_rows
11,11


Markdown Cell — Explanation

This data simulates:

U1001 → Highly engaged (multiple features, multiple sessions)

U1002 → Moderate engagement

U1003 → Very low engagement

U1004 → Trial user with limited interaction

This variation is important so churn scoring logic produces different risk levels.

In [0]:
%sql
CREATE TABLE IF NOT EXISTS saas_bronze.subscription_billing (
    user_id STRING,
    subscription_id STRING,
    plan_type STRING,
    subscription_status STRING,
    amount DOUBLE,
    billing_date DATE,
    renewal_date DATE
)
USING DELTA;


This table stores subscription lifecycle data:

Plan type (Basic, Pro)

Subscription state (Trial, Active, Churned)

Billing amounts and renewal dates

This dataset helps track customer lifecycle changes.

In [0]:
%sql
INSERT INTO saas_bronze.subscription_billing VALUES
('U1001','SUB1001','Pro','Active',799,'2026-01-01','2026-02-01'),
('U1002','SUB1002','Basic','Active',299,'2026-01-01','2026-02-01'),
('U1003','SUB1003','Pro','Churned',799,'2025-12-01','2026-01-01'),
('U1004','SUB1004','Basic','Trial',0,'2026-01-01','2026-01-15');


num_affected_rows,num_inserted_rows
4,4


This represents subscription lifecycle states:

Active users → U1001, U1002

Churned user → U1003

Trial user → U1004

In [0]:
%sql
CREATE TABLE IF NOT EXISTS saas_bronze.support_ticket_history (
    ticket_id STRING,
    user_id STRING,
    issue_category STRING,
    ticket_text STRING,
    created_at TIMESTAMP,
    ticket_status STRING
)
USING DELTA;


This table captures customer support interactions.

It will later be used to:

Perform sentiment analysis

Identify frustrated customers

Improve churn prediction accuracy

In [0]:
%sql
INSERT INTO saas_bronze.support_ticket_history VALUES
('T1001','U1001','Feature Request','Love the analytics dashboard','2026-01-02 12:00:00','Closed'),

('T1002','U1002','Billing','Payment failed twice','2026-01-03 13:00:00','Open'),

('T1003','U1003','Performance','Very bad performance and slow loading','2026-01-04 15:00:00','Closed'),

('T1004','U1004','Login','App not working properly','2026-01-05 18:00:00','Open');


num_affected_rows,num_inserted_rows
4,4


This creates sentiment diversity:

U1001 → Positive/Neutral ticket

U1002 → Billing issue (Negative)

U1003 → Strong negative complaint

U1004 → Technical frustration

This ensures sentiment classification impacts churn risk realistically.

SILVER LAYER

In [0]:
%sql
CREATE OR REPLACE TABLE saas_silver.session_quality AS
SELECT
    user_id,
    session_id,
    COUNT(DISTINCT feature_name) AS distinct_features,
    COUNT(*) AS total_events,
    ROUND(COUNT(DISTINCT feature_name)*1.0 / COUNT(*),2) AS session_quality_score
FROM saas_bronze.feature_usage_logs
GROUP BY user_id, session_id;


num_affected_rows,num_inserted_rows


Instead of measuring engagement by raw clicks, we calculate:

Session Quality Score = distinct features used / total events

this,

Prevents bias from repetitive clicks

Rewards feature diversity

Reflects meaningful engagement

This normalizes user activity properly.

In [0]:
%sql
CREATE OR REPLACE TABLE saas_silver.ticket_sentiment AS
SELECT
    ticket_id,
    user_id,
    ticket_text,
    CASE
        WHEN LOWER(ticket_text) LIKE '%not working%'
          OR LOWER(ticket_text) LIKE '%failed%'
          OR LOWER(ticket_text) LIKE '%bad%'
          OR LOWER(ticket_text) LIKE '%issue%'
        THEN 'Negative'
        ELSE 'Neutral'
    END AS sentiment
FROM saas_bronze.support_ticket_history;


num_affected_rows,num_inserted_rows


This performs rule-based sentiment analysis:

Detects negative language

Classifies tickets as Negative or Neutral

This helps identify frustrated customers.

In production, this could be replaced with ML/NLP.

In [0]:
%sql
CREATE OR REPLACE TABLE saas_silver.customer_profile AS
SELECT
    b.user_id,
    b.plan_type,
    b.subscription_status,
    AVG(s.session_quality_score) AS avg_session_quality,
    COUNT(DISTINCT t.ticket_id) AS total_tickets,
    SUM(CASE WHEN ts.sentiment='Negative' THEN 1 ELSE 0 END) AS negative_tickets
FROM saas_bronze.subscription_billing b
LEFT JOIN saas_silver.session_quality s
    ON b.user_id = s.user_id
LEFT JOIN saas_bronze.support_ticket_history t
    ON b.user_id = t.user_id
LEFT JOIN saas_silver.ticket_sentiment ts
    ON b.user_id = ts.user_id
GROUP BY b.user_id, b.plan_type, b.subscription_status;


num_affected_rows,num_inserted_rows


This creates a 360° unified customer view by combining:

Behavioral engagement

Subscription state

Support interactions

This table becomes the foundation for churn analysis.

In [0]:
%sql
MERGE INTO saas_silver.customer_profile AS target
USING saas_bronze.subscription_billing AS source
ON target.user_id = source.user_id
WHEN MATCHED THEN
UPDATE SET target.subscription_status = source.subscription_status
WHEN NOT MATCHED THEN
INSERT (user_id, plan_type, subscription_status)
VALUES (source.user_id, source.plan_type, source.subscription_status);


num_affected_rows,num_updated_rows,num_deleted_rows,num_inserted_rows
4,4,0,0


This implements Change Data Capture (CDC):

Detects subscription status updates

Updates existing customers

Inserts new ones

This ensures subscription lifecycle states remain current.

GOLD LAYER

In [0]:
%sql
CREATE OR REPLACE TABLE saas_gold.customer_churn_risk AS
SELECT
    user_id,
    plan_type,
    subscription_status,
    avg_session_quality,
    total_tickets,
    negative_tickets,
    CASE
        WHEN subscription_status='Churned' THEN 100
        WHEN avg_session_quality < 0.3 AND negative_tickets > 0 THEN 85
        WHEN avg_session_quality < 0.5 THEN 65
        WHEN negative_tickets > 0 THEN 55
        ELSE 20
    END AS churn_risk_score
FROM saas_silver.customer_profile;


num_affected_rows,num_inserted_rows


This generates a churn risk score (1–100):

High risk if:

Already churned

Low engagement

Negative support tickets

This is the core business output.

In [0]:
%sql
CREATE OR REPLACE TABLE saas_gold.feature_roi_table AS
SELECT
    f.feature_name,
    COUNT(DISTINCT f.user_id) AS users_using_feature,
    COUNT(DISTINCT CASE
        WHEN c.subscription_status='Active'
        THEN f.user_id
    END) AS retained_users,
    ROUND(
      COUNT(DISTINCT CASE WHEN c.subscription_status='Active'
      THEN f.user_id END) * 1.0 /
      COUNT(DISTINCT f.user_id), 2
    ) AS retention_ratio
FROM saas_bronze.feature_usage_logs f
LEFT JOIN saas_silver.customer_profile c
    ON f.user_id = c.user_id
GROUP BY f.feature_name;


num_affected_rows,num_inserted_rows


This table answers:

👉 Which features drive retention?

Retention ratio shows how many feature users remain active.

This helps optimize product investment decisions.

PRODUCTION OPTIMIZATION

In [0]:
%sql
OPTIMIZE saas_bronze.subscription_billing;


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, null, null, 0, 0, 1, 1, true, 0, 0, 1771428182099, 1771428182953, 8, 0, null, List(0, 0), null, 7, 7, 0, 0, null, null)"


Since Serverless does not support table caching, we use Delta table optimization instead.

OPTIMIZE:

Compacts small files

Improves query scan performance

Reduces I/O cost

This is the correct production optimization strategy for Serverless environments.

In [0]:
%sql
OPTIMIZE saas_bronze.subscription_billing
ZORDER BY (user_id);


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 2158), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1771428230639, 1771428231255, 8, 0, null, List(0, 0), null, 7, 7, 0, 0, null, null)"


Z-Ordering improves query performance when filtering or joining on user_id.

Since most joins in this project use user_id, this significantly speeds up:

Customer profile joins

Churn risk calculation

Feature ROI analysis

This replaces the need for caching.

Z-ordering

In [0]:
%sql
OPTIMIZE saas_silver.customer_profile
ZORDER BY (user_id);


path,metrics
,"List(0, 0, List(null, null, 0.0, 0, 0), List(null, null, 0.0, 0, 0), 0, List(minCubeSize(107374182400), List(0, 0), List(1, 2098), 0, List(0, 0), 0, null), null, 0, 0, 1, 1, false, 0, 0, 1771428271319, 1771428272222, 8, 0, null, List(0, 0), null, 6, 6, 0, 0, null, null)"


In [0]:
%sql
SELECT /*+ BROADCAST(b) */
    f.user_id,
    f.feature_name,
    b.plan_type
FROM saas_bronze.feature_usage_logs f
JOIN saas_bronze.subscription_billing b
ON f.user_id = b.user_id;


user_id,feature_name,plan_type
U1001,Dashboard,Pro
U1001,Reports,Pro
U1001,Analytics,Pro
U1001,Dashboard,Pro
U1001,Settings,Pro
U1002,Dashboard,Basic
U1002,Dashboard,Basic
U1002,Reports,Basic
U1003,Dashboard,Pro
U1004,Reports,Basic


Broadcast join forces Spark to:

Send small billing table to all nodes

Avoid shuffle-heavy joins

Use this when:

One table is significantly smaller than the other

This improves multi-way join performance.

In [0]:
%sql
CREATE OR REPLACE VIEW saas_gold.masked_customer_view AS
SELECT
    sha2(user_id,256) AS masked_user_id,
    churn_risk_score,
    plan_type,
    subscription_status
FROM saas_gold.customer_churn_risk;


This implements data security:

Masks user identifiers

Preserves analytical usability

Prevents exposure of sensitive data

This is required for compliance and production readiness.

Churn Risk Distribution



In [0]:
%sql
SELECT
    CASE
        WHEN churn_risk_score >= 80 THEN 'High Risk'
        WHEN churn_risk_score >= 50 THEN 'Medium Risk'
        ELSE 'Low Risk'
    END AS risk_category,
    COUNT(*) AS user_count
FROM saas_gold.customer_churn_risk
GROUP BY risk_category;


risk_category,user_count
Low Risk,1
Medium Risk,2
High Risk,1


Databricks visualization. Run in Databricks to view.

Negative Tickets vs Subscription Status
🎯 Purpose:

See if frustrated users churn more.

In [0]:
%sql
SELECT
    subscription_status,
    SUM(negative_tickets) AS total_negative_tickets
FROM saas_silver.customer_profile
GROUP BY subscription_status;


subscription_status,total_negative_tickets
Active,2
Churned,1
Trial,1


Databricks visualization. Run in Databricks to view.

Feature Retention Ratio


This graph highlights which product features correlate with higher retention. These features should receive more investment

In [0]:
%sql
SELECT
    feature_name,
    retention_ratio
FROM saas_gold.feature_roi_table
ORDER BY retention_ratio DESC;


feature_name,retention_ratio
Settings,1.00
Analytics,1.00
Dashboard,0.67
Reports,0.67


Databricks visualization. Run in Databricks to view.